**1. Représenter le Sudoku comme un CSP (Problème de Satisfaction de Contraintes)**
1. Variables : Dans un Sudoku, chaque case peut être considérée comme une variable. Pour un Sudoku standard de 9x9, vous aurez 81 variables, généralement nommées par leur position dans la grille (par exemple, (i,j) pour la case de la i-ème ligne et la j-ème colonne).

2. Domaines : Le domaine de chaque variable dans le Sudoku est l'ensemble des chiffres de 1 à 9, car chaque case peut contenir n'importe lequel de ces chiffres.

3. Contraintes : Les contraintes du Sudoku sont que chaque chiffre doit apparaître exactement une fois par ligne, une fois par colonne et une fois par sous-grille 3x3. Ces contraintes peuvent être représentées comme suit :

 * Contrainte de ligne : Aucun chiffre répété dans une même ligne.
 * Contrainte de colonne : Aucun chiffre répété dans une même colonne.
 * Contrainte de sous-grille 3x3 : Aucun chiffre répété dans une même sous-grille 3x3.

In [ ]:
class SudokuCSP:
    def __init__(self, initial_state):
        self.variables = [(i, j) for i in range(9) for j in range(9)]
        self.domains = {var: list(range(1, 10)) if initial_state[var[0]][var[1]] == 0 else [initial_state[var[0]][var[1]]] for var in self.variables}
        self.constraints = []
        self.initialize_constraints()

    def initialize_constraints(self):
        # Add constraints for rows, columns, and 3x3 squares
        for i in range(9):
            for j in range(9):
                # Row and column constraints
                for k in range(9):
                    if k != j:
                        self.constraints.append(((i, j), (i, k)))
                    if k != i:
                        self.constraints.append(((i, j), (k, j)))

                # 3x3 square constraints
                for k in range(3):
                    for l in range(3):
                        row = 3 * (i // 3) + k
                        col = 3 * (j // 3) + l
                        if (row, col) != (i, j):
                            self.constraints.append(((i, j), (row, col)))

    def assign_initial_state(self, initial_state):
        # Assign initial values and adjust domains
        for i in range(9):
            for j in range(9):
                if initial_state[i][j] != 0:
                    self.domains[(i, j)] = [initial_state[i][j]]

    def is_assignment_valid(self, board, row, col, num):
        # Check if a number is valid in the given row, column and 3x3 square
        for x in range(9):
            if board[row][x] == num or board[x][col] == num or \
               board[3 * (row // 3) + x // 3][3 * (col // 3) + x % 3] == num:
                return False
        return True
    def count_possible_values(self, board, row, col):
        possible_values = set(range(1, 10))  # Chiffres possibles de 1 à 9
        for i in range(9):
            # Enlever les valeurs déjà présentes dans la ligne, colonne et sous-grille
            possible_values.discard(board[row][i])
            possible_values.discard(board[i][col])
            possible_values.discard(board[3 * (row // 3) + i // 3][3 * (col // 3) + i % 3])
        return len(possible_values)


**2. Résolution du Sudoku par l'algorithme de Backtracking**

In [ ]:
def solve_sudoku(board, sudoku_csp):
    empty = find_empty_location(board)
    if not empty:
        return True  # No empty space, Sudoku solved
    row, col = empty

    for num in range(1, 10):
        if sudoku_csp.is_assignment_valid(board, row, col, num):
            board[row][col] = num
            if solve_sudoku(board, sudoku_csp):
                return True
    board[row][col] = 0  # Backtrack

    return False
def find_empty_location(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:  # 0 signifie une case vide
                return (i, j)
    return None

In [ ]:
sudoku_board = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]
sudoku_csp = SudokuCSP(sudoku_board)

if solve_sudoku(sudoku_board, sudoku_csp):
    for row in sudoku_board:
        print(row)
else:
    print("Pas de solution")

[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]


**3. Amélioration de la performance de l'algorithme avec une heuristique**

L'heuristique "Minimum Remaining Values" (MRV) peut être utilisée pour améliorer l'efficacité de l'algorithme de backtracking. L'idée est de choisir en premier la case avec le moins de valeurs possibles. Ceci réduit l'espace de recherche et mène souvent à une solution plus rapidement.

* Heuristique MRV : Parmi toutes les cases vides, choisir celle qui a le moins de valeurs possibles dans son domaine.
* Intégration avec Backtracking : Utiliser cette case pour continuer la résolution avec l'algorithme de backtracking.



In [ ]:
def find_least_remaining_values(board, sudoku_csp):
    min_count = 10  # Plus que le nombre maximal de valeurs possibles (9)
    min_pos = None
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:  # Si la case est vide
                count = sudoku_csp.count_possible_values(board, i, j)
                if count < min_count:
                    min_count = count
                    min_pos = (i, j)
    return min_pos


In [ ]:
def solve_sudoku_with_mrv(board, sudoku_csp):
    empty = find_least_remaining_values(board, sudoku_csp)
    if empty is None:
        return True  # Sudoku résolu
    row, col = empty

    for num in range(1, 10):
        if sudoku_csp.is_assignment_valid(board, row, col, num):
            board[row][col] = num
            if solve_sudoku(board, sudoku_csp):
                return True
    board[row][col] = 0
    return False


In [ ]:
# Exemple de grille Sudoku
sudoku_board = [
    [0, 6, 0, 1, 0, 4, 0, 5, 0],
    [0, 0, 8, 3, 0, 5, 6, 0, 0],
    [2, 0, 0, 0, 0, 0, 0, 0, 1],
    [8, 0, 0, 4, 0, 7, 0, 0, 6],
    [0, 0, 6, 0, 0, 0, 3, 0, 0],
    [7, 0, 0, 9, 0, 1, 0, 0, 4],
    [5, 0, 0, 0, 0, 0, 0, 0, 2],
    [0, 0, 7, 2, 0, 6, 9, 0, 0],
    [0, 4, 0, 5, 0, 8, 0, 7, 0]
]
sudoku_csp = SudokuCSP(sudoku_board)

if solve_sudoku_with_mrv(sudoku_board, sudoku_csp):
    for row in sudoku_board:
        print(row)
else:
    print("Pas de solution")


[9, 6, 3, 1, 7, 4, 2, 5, 8]
[1, 7, 8, 3, 2, 5, 6, 4, 9]
[2, 5, 4, 6, 8, 9, 7, 3, 1]
[8, 2, 1, 4, 3, 7, 5, 9, 6]
[4, 9, 6, 8, 5, 2, 3, 1, 7]
[7, 3, 5, 9, 6, 1, 8, 2, 4]
[5, 8, 9, 7, 1, 3, 4, 6, 2]
[3, 1, 7, 2, 4, 6, 9, 8, 5]
[6, 4, 2, 5, 9, 8, 1, 7, 3]


In [ ]:
def find_least_constraining_value(board, sudoku_csp, row, col):
    values = sudoku_csp.domains[(row, col)]
    count_dict = {}

    for value in values:
        count = 0
        for i in range(9):
            if board[row][i] == 0 and value in sudoku_csp.domains[(row, i)]:
                count += 1
            if board[i][col] == 0 and value in sudoku_csp.domains[(i, col)]:
                count += 1
            subgrid_row = 3 * (row // 3) + i // 3
            subgrid_col = 3 * (col // 3) + i % 3
            if board[subgrid_row][subgrid_col] == 0 and value in sudoku_csp.domains[(subgrid_row, subgrid_col)]:
                count += 1

        count_dict[value] = count

    # Sort values based on the count of constraints they impose on other variables
    sorted_values = sorted(count_dict, key=lambda k: count_dict[k])

    return sorted_values


def solve_sudoku_with_lcv(board, sudoku_csp):
    empty = find_least_remaining_values(board, sudoku_csp)
    if empty is None:
        return True  # Sudoku solved
    row, col = empty

    values = find_least_constraining_value(board, sudoku_csp, row, col)

    for num in values:
        if sudoku_csp.is_assignment_valid(board, row, col, num):
            board[row][col] = num
            if solve_sudoku_with_lcv(board, sudoku_csp):
                return True
    board[row][col] = 0

    return False

In [ ]:
# Exemple de grille Sudoku
sudoku_board = [
    [0, 6, 0, 1, 0, 4, 0, 5, 0],
    [0, 0, 8, 3, 0, 5, 6, 0, 0],
    [2, 0, 0, 0, 0, 0, 0, 0, 1],
    [8, 0, 0, 4, 0, 7, 0, 0, 6],
    [0, 0, 6, 0, 0, 0, 3, 0, 0],
    [7, 0, 0, 9, 0, 1, 0, 0, 4],
    [5, 0, 0, 0, 0, 0, 0, 0, 2],
    [0, 0, 7, 2, 0, 6, 9, 0, 0],
    [0, 4, 0, 5, 0, 8, 0, 7, 0]
]
sudoku_csp = SudokuCSP(sudoku_board)

if solve_sudoku_with_lcv(sudoku_board, sudoku_csp):
    for row in sudoku_board:
        print(row)
else:
    print("Pas de solution")


[9, 6, 3, 1, 7, 4, 2, 5, 8]
[1, 7, 8, 3, 2, 5, 6, 4, 9]
[2, 5, 4, 6, 8, 9, 7, 3, 1]
[8, 2, 1, 4, 3, 7, 5, 9, 6]
[4, 9, 6, 8, 5, 2, 3, 1, 7]
[7, 3, 5, 9, 6, 1, 8, 2, 4]
[5, 8, 9, 7, 1, 3, 4, 6, 2]
[3, 1, 7, 2, 4, 6, 9, 8, 5]
[6, 4, 2, 5, 9, 8, 1, 7, 3]


In [ ]:
import time

# Function to measure time and solve Sudoku
def measure_time_and_solve(solver_function, board, sudoku_csp):
    start_time = time.time()
    result = solver_function(board, sudoku_csp)
    end_time = time.time()
    elapsed_time = end_time - start_time

    if result:
        print(f"Sudoku solved in {elapsed_time:.6f} seconds")
    else:
        print("Unable to solve Sudoku")



# Measure time for solve_sudoku
measure_time_and_solve(solve_sudoku, sudoku_board.copy(), sudoku_csp)

# Measure time for solve_sudoku_with_mrv
measure_time_and_solve(solve_sudoku_with_mrv, sudoku_board.copy(), sudoku_csp)

# Measure time for solve_sudoku_with_lcv
measure_time_and_solve(solve_sudoku_with_lcv, sudoku_board.copy(), sudoku_csp)


Sudoku solved in 0.000020 seconds
Sudoku solved in 0.000018 seconds
Sudoku solved in 0.000018 seconds
